In [3]:
import os
import requests
import gzip

# --- PASO 1: CONFIGURACIÓN INICIAL ---
# Define el código PDB y el nombre del residuo del ligando
pdb_code = '4PYP'
ligand_resname = 'BNG'
receptor_chain = 'A'

# Define los directorios de trabajo
working_dir = f"run/{pdb_code}_smina"
os.makedirs(working_dir, exist_ok=True)
print(f"Directorio de trabajo creado: {working_dir}")

# --- PASO 2: DESCARGA DE LA ESTRUCTURA PDB ---
# Descarga el archivo PDB de RCSB.org
pdb_url = f"https://files.rcsb.org/download/{pdb_code}.pdb.gz"
pdb_gz_path = os.path.join(working_dir, f"{pdb_code}.pdb.gz")
pdb_path = os.path.join(working_dir, f"{pdb_code}.pdb")

print(f"\nDescargando {pdb_code}.pdb.gz...")
try:
    response = requests.get(pdb_url)
    if response.status_code == 200:
        with open(pdb_gz_path, 'wb') as f:
            f.write(response.content)
        # Descomprime el archivo
        with gzip.open(pdb_gz_path, 'rb') as f_in:
            with open(pdb_path, 'wb') as f_out:
                f_out.write(f_in.read())
        print("Descarga y descompresión exitosa.")
        os.remove(pdb_gz_path)
    else:
        print(f"Error al descargar la estructura: {response.status_code}")
except Exception as e:
    print(f"Ocurrió un error durante la descarga: {e}")

# --- PASO 3: PREPARACIÓN DEL RECEPTOR Y EL LIGANDO ---
# Creamos las rutas de los archivos de salida
receptor_pdb = os.path.join(working_dir, f"{pdb_code}_{receptor_chain}.pdb")
ligand_pdb = os.path.join(working_dir, f"{pdb_code}_{ligand_resname}.pdb")
receptor_clean_pdb = os.path.join(working_dir, f"{pdb_code}_clean.pdb")

# Extraemos la cadena del receptor y el ligando usando 'grep'
print("\nPreparando archivos de receptor y ligando...")
os.system(f"grep ' {receptor_chain} ' {pdb_path} > {receptor_pdb}")
os.system(f"grep ' {ligand_resname} ' {receptor_pdb} > {ligand_pdb}")
os.system(f"grep -v HETATM {receptor_pdb} > {receptor_clean_pdb}")

# Verificamos si los archivos se crearon correctamente
if not os.path.exists(ligand_pdb) or os.path.getsize(ligand_pdb) == 0:
    print(f"Error: No se pudo encontrar el ligando '{ligand_resname}' en el archivo PDB.")
    # El script termina aquí si no hay ligando.
else:
    print("Archivos de receptor y ligando preparados con éxito.")

# --- PASO 4: DOCKING RÍGIDO CON SMINA ---
print("\n--- Ejecutando Docking Rígido con Smina ---")

docking_rigid_output = os.path.join(working_dir, f"{pdb_code}_docking.sdf")

# El comando de Smina usa la opción --autobox_ligand para definir el sitio de unión
# de forma automática basado en las coordenadas del ligando de referencia.
smina_rigid_cmd = (
    f"smina --cpu 4 --seed 0 --exhaustiveness 32 "
    f"--autobox_add +6 --autobox_ligand {ligand_pdb} "
    f"-r {receptor_clean_pdb} -l {ligand_pdb} -o {docking_rigid_output}"
)

os.system(smina_rigid_cmd)
print("\nDocking rígido completado. Resultados en: {docking_rigid_output}")

# --- PASO 5: DOCKING FLEXIBLE CON SMINA (Opcional) ---
print("\n--- Ejecutando Docking Flexible con Smina ---")

docking_flex_output = os.path.join(working_dir, f"{pdb_code}_flex_docking.sdf")
flex_residues_output = os.path.join(working_dir, f"{pdb_code}_flex_residues.pdb")

# Smina utiliza --flexdist_ligand y --flexdist para definir los residuos flexibles.
smina_flex_cmd = (
    f"smina --cpu 4 --seed 0 --exhaustiveness 64 "
    f"--flexdist_ligand {ligand_pdb} --flexdist 4 "
    f"--autobox_add +8 --autobox_ligand {ligand_pdb} "
    f"-r {receptor_clean_pdb} -l {ligand_pdb} -o {docking_flex_output} "
    f"--out_flex {flex_residues_output}"
)

os.system(smina_flex_cmd)
print("\nDocking flexible completado. Resultados en: {docking_flex_output}")
print("Residuos flexibles guardados en: {flex_residues_output}")

Directorio de trabajo creado: run/4PYP_smina

Descargando 4PYP.pdb.gz...
Descarga y descompresión exitosa.

Preparando archivos de receptor y ligando...
Archivos de receptor y ligando preparados con éxito.

--- Ejecutando Docking Rígido con Smina ---
   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)
1.923        num_tors_div

Using random seed: 0

0%   10   20   30   40   50   60   70   